# Test creation

This notebook will create the necessary folder structure for your tests. 
See more documentation in: https://docs.smartcitizen.me/Guides/Organise%20your%20data/

## Test information
Add the information from your test. Some inputs are given below as an example.

In [ ]:
from src.data.test import test_wrapper
import pandas as pd
from src.saf import std_out

### ---------------------------------------
### ------------INPUT METADATA-------------
### ---------------------------------------

# Date when you performed the test in 'YYYY-MM-DD' format
date = '2020-03-26'
# INTernal or EXTernal test (made by you or others)
who = 'INT' 
# Short title for the test
short_name = 'AMAZING_TEST'
# Comment for your test. Make this as long as you need to describe fully the purpose of this test
comment = ''
# Project
project = 'SmartCitizen'
# Firmware version
commit = ''
# Who made the test
author = 'Myself'
# Test type (indoor, outdoor, anything that helps you organise later on)
type_test = 'outdoor'
# If you are going to document the results somewhere, you can put a link below
report = ''
notes = ''

### ---------------------------------------
### ----------END INPUT METADATA-----------
### ---------------------------------------

# Create test object
newtest = test_wrapper(pd.to_datetime(date).strftime('%Y-%m') + '_' + who + '_' + short_name)

# Add General test details
details = {'project': project, 
           'commit': commit, 
           'author': author, 
           'type_test': type_test, 
           'report': report, 
           'comment': comment,
           'notes': notes}

# Output test name
std_out(f'New test name: {newtest.name}', 'SUCCESS')

## Add devices
Here we will add the devices for our test. The devices must be an instanc of `src.data.device`. 
There are plenty of blueprints already for each device available at `data/interim/blueprints.yml` . If not filled in, the blueprint default value will be used.

In [ ]:
from src.data.device import device_wrapper

STATIONS = {'10752':{'CO':162700911,'NO2':202702041,'OX':204702949},
            '10751':{'CO':162700906,'NO2':202702038,'OX':204702950}}

devices = list()

for device_name in STATIONS.keys():
    # Example of a sck_21_station from the SC API, using three electrochemical sensors with given ids
    devices.append(device_wrapper(blueprint = 'sck_21_station', descriptor = {'id': '10752', 
                                                                             'source': 'api',
                                                                             'metrics': {'CO': {'id': 2168293},
                                                                                         'NO2': {'id': 2168294},
                                                                                         'O3': {'id': 2168295}
                                                                                        },
                                                                              'frequency': '1Min'
                                                                             }))
    
    # Example of a sck_20_station from the local csv data. In this case no alphasense data is specified
    devices.append(device_wrapper(blueprint = 'sck_20_station', descriptor = {'id': '5261',
                                                                             'source': 'csv',
                                                                             'frequency': '1Min',
                                                                             'location': 'Europe/Madrid',                                                                              
                                                                             'raw_data_file': '5261.csv'}))


## Process the test
This will generate a folder under `data/processed/YYYY/MM/<test_name>` and add raw files (if any), load csv files, convert units and names, resample according to frequency, and export accordingly. The test and devices description will then be dumped into `test_description.yaml`.

If source is from API, it will add the information in the `test_description.yaml` for later download.

In [ ]:
# Add details
newtest.add_details(details)
# Add devices
for device in devices: newtest.add_device(device)
# Process data
newtest.make()